# Setup and Load dataset



In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
!gdown --id 1819QbWTepXZzOXREC1nPaBhAP8Ww3rW2

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1819QbWTepXZzOXREC1nPaBhAP8Ww3rW2
To: /content/Cranfield Dataset NLP.zip
100% 499k/499k [00:00<00:00, 113MB/s]


In [ ]:
!unzip -o "Cranfield Dataset NLP.zip"  -d  "/content"

Archive:  Cranfield Dataset NLP.zip
   creating: /content/cranfield/
  inflating: /content/cranfield/.DS_Store  
   creating: /content/__MACOSX/
   creating: /content/__MACOSX/cranfield/
  inflating: /content/__MACOSX/cranfield/._.DS_Store  
  inflating: /content/cranfield/cran_queries.json  
  inflating: /content/__MACOSX/cranfield/._cran_queries.json  
  inflating: /content/cranfield/cran_qrels.json  
  inflating: /content/__MACOSX/cranfield/._cran_qrels.json  
  inflating: /content/cranfield/cran_docs.json  
  inflating: /content/__MACOSX/cranfield/._cran_docs.json  
  inflating: /content/cranfield/README.txt  
  inflating: /content/__MACOSX/cranfield/._README.txt  


## Data Loading

In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

#### Name of the file which contain all the item properties

In [ ]:

import json
with open('cranfield/cran_docs.json', 'r') as datafile:
        data = json.load(datafile)
content_df = pd.DataFrame(data)

In [ ]:
content_df.head(1)

,id,author,bibliography,body,title
0,1,"brenckman,m.","j. ae. scs. 25, 1958, 324.",experimental investigation of the aerodynamics...,experimental investigation of the aerodynamics...


In [ ]:
import json
with open('cranfield/cran_qrels.json', 'r') as datafile:
        data = json.load(datafile)
qrels_df = pd.DataFrame(data)

In [ ]:
qrels_df.head(15)

,query_num,position,id
0,1,2,184
1,1,2,29
2,1,2,31
3,1,3,12
4,1,3,51
5,1,3,102
6,1,4,13
7,1,4,14
8,1,4,15
9,1,2,57


In [ ]:
import json
with open('cranfield/cran_queries.json', 'r') as datafile:
        data = json.load(datafile)
queries_df = pd.DataFrame(data)

In [ ]:
queries_df.head(1)

,query number,query
0,1,what similarity laws must be obeyed when const...


In [ ]:
all_queries=list(queries_df['query'].values)

## Details about dataset

In [ ]:
itemid="id"

In [ ]:
features=['title','body']

In [ ]:
allcols=[itemid]
for i in features:
  allcols.append(i)

# Setup

In [ ]:
content_df['NewTag']=""
for i in features:
  content_df[i] = content_df[i].fillna(' ')
for i in features:
  content_df['NewTag']+=(" "+content_df[i])
content_df['NewTag']=content_df['NewTag'].astype(str)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [ ]:
def clean_text(text):
    
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub( re.compile("\'s"), ' ', text)
    text = re.sub(re.compile("\\r\\n"), ' ', text)
    text = re.sub(re.compile(r"[^\w\s]"), ' ', text)
    return text

In [ ]:
stopwords=set(stopwords.words('english'))

In [ ]:
content_df['clean'] = content_df['NewTag'].apply(clean_text)
# content_df['token_lem_sentence'] = content_df['clean'].apply(
#         lambda x: tokenizer(x))

In [ ]:
def reduce(text):
        """
		Stemming/Lemmatization
		Parameters
		----------
		arg1 : list
			A list of lists where each sub-list a sequence of tokens
			representing a sentence
		Returns
		-------
		list
			A list of lists where each sub-list is a sequence of
			stemmed/lemmatized tokens representing a sentence
		"""
        sno = nltk.stem.SnowballStemmer('english')
        reducedText = ''
        for word in text.split():
          
            reducedText +=(' '+(sno.stem(word)))
        return reducedText

In [ ]:
content_df['clean'] = content_df['clean'].apply(reduce)

In [ ]:
content_df.head(1)

,id,author,bibliography,body,title,NewTag,clean
0,1,"brenckman,m.","j. ae. scs. 25, 1958, 324.",experimental investigation of the aerodynamics...,experimental investigation of the aerodynamics...,experimental investigation of the aerodynamic...,experiment investig of the aerodynam of a win...


# Model train

In [ ]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
tfidf_tran= vectorizer.fit_transform(content_df['clean'] )

In [ ]:
import pickle
with open('tf-idf.pkl','wb') as handle:
    pickle.dump(tfidf_tran, handle)

In [ ]:
tfidf_tran = pickle.load(open('tf-idf.pkl','rb'))

# Sample search 

In [ ]:
query = all_queries[0]
print(query)
query =clean_text(query)
query = reduce(query)

query_vec = vectorizer.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
results = cosine_similarity(tfidf_tran,query_vec).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc
# Print Top 10 results
ids=[]
for i in results.argsort()[-10:][::-1]:
  ids.append( content_df.iloc[i][itemid])
content_df[content_df[itemid].isin(ids)][allcols]

what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .


,id,title,body
11,12,some structural and aerelastic considerations ...,some structural and aerelastic considerations ...
12,13,similarity laws for stressing heated wings .,similarity laws for stressing heated wings . ...
50,51,theory of aircraft structural models subjected...,theory of aircraft structural models subjected...
55,56,an analysis of the applicability of the hypers...,an analysis of the applicability of the hypers...
183,184,scale models for thermo-aeroelastic research .,scale models for thermo-aeroelastic research ....
358,359,note on the hypersonic similarity law for an u...,note on the hypersonic similarity law for an u...
434,435,application of similar solutions to calculatio...,application of similar solutions to calculatio...
485,486,similarity laws for aerothermoelastic testing .,similarity laws for aerothermoelastic testing ...
745,746,aeroelastic problems in connection with high s...,aeroelastic problems in connection with high s...
874,875,models for aeroelastic investigation .,models for aeroelastic investigation . this ...


# Evaluation

In [ ]:
queries_df.columns

Index(['query number', 'query'], dtype='object')

In [ ]:
str(all_queries[0])

'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .'

In [ ]:
qrels_df['id']=qrels_df['id'].astype(str)
content_df[itemid]=content_df[itemid].astype(str)

## TOP SEARCH RESULT

## How many results search result at first rank gave complete answer to search query

### Simple semantic search

In [ ]:
topk=1
totalhit=0
for qry in all_queries:
  query_id=list(queries_df[queries_df['query']==qry]['query number'])[0]

  query =clean_text(qry)
  query = reduce(qry)

  query_vec = vectorizer.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
  results = cosine_similarity(tfidf_tran,query_vec).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc
  # Print Top 10 results
  ids=[]
  for i in results.argsort()[-1:][::-1]:
    ids.append( content_df.iloc[i][itemid])

  results = ids

  actuall=list(qrels_df[(qrels_df['query_num']==str(query_id) )& (qrels_df['id'].isin(results)) & (qrels_df['position']<=1) ][itemid])

  totalhit+=len(actuall)




In [ ]:
pre=totalhit/len(all_queries)
print('precision@1 :'+str(pre))

precision@1 :0.41333333333333333


## How many results search result at first rank gave high degree relivent answer to search query

### Simple  search

In [ ]:
topk=1
totalhit=0
for qry in all_queries:
  query_id=list(queries_df[queries_df['query']==qry]['query number'])[0]

  query =clean_text(qry)
  query = reduce(qry)

  query_vec = vectorizer.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
  results = cosine_similarity(tfidf_tran,query_vec).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc
  # Print Top 10 results
  ids=[]
  for i in results.argsort()[-1:][::-1]:
    ids.append( content_df.iloc[i][itemid])

  results = ids

  actuall=list(qrels_df[(qrels_df['query_num']==str(query_id) )& (qrels_df['id'].isin(results)) & (qrels_df['position']<=2) ][itemid])

  totalhit+=len(actuall)




In [ ]:
pre=totalhit/len(all_queries)
print('precision@1 :'+str(pre))

precision@1 :0.5155555555555555


## How many results search result at first rank gave useful answer to search query

### Simple  search

In [ ]:
topk=1
totalhit=0
for qry in all_queries:
  query_id=list(queries_df[queries_df['query']==qry]['query number'])[0]

  query =clean_text(qry)
  query = reduce(qry)

  query_vec = vectorizer.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
  results = cosine_similarity(tfidf_tran,query_vec).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc
  # Print Top 10 results
  ids=[]
  for i in results.argsort()[-1:][::-1]:
    ids.append( content_df.iloc[i][itemid])

  results = ids

  actuall=list(qrels_df[(qrels_df['query_num']==str(query_id) )& (qrels_df['id'].isin(results)) & (qrels_df['position']<=3) ][itemid])

  totalhit+=len(actuall)




In [ ]:
pre=totalhit/len(all_queries)
print('precision@1 :'+str(pre))

precision@1 :0.6622222222222223


## TOP 10  SEARCH RESULT

## How many results search result at first rank gave usefull answer to search query

### Simple semantic search

In [ ]:
topk=10
totalhit=0
for qry in all_queries:
  query_id=list(queries_df[queries_df['query']==qry]['query number'])[0]

  query =clean_text(qry)
  query = reduce(qry)

  query_vec = vectorizer.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
  results = cosine_similarity(tfidf_tran,query_vec).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc
  # Print Top 10 results
  ids=[]
  for i in results.argsort()[-10:][::-1]:
    ids.append( content_df.iloc[i][itemid])

  results = ids

  actuall=list(qrels_df[(qrels_df['query_num']==str(query_id) )& (qrels_df['id'].isin(results)) & (qrels_df['position']<=3) ][itemid])

  totalhit+=len(actuall)




In [ ]:
pre=totalhit/(len(all_queries)*10)
print('precision@10 :'+str(pre))

precision@10 :0.2604444444444444
